<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/characters/novel_collecting/chatbot%E7%9A%84embedding%E6%8A%BD%E5%8F%96%E5%92%8Cjsonl%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

implemented by 李鲁鲁

- [x] 编写针对embedding的base64压缩，解压缩函数
- [x] 解压缩一个角色的文字包
- [x] clone Haruhi2 进行抽取
- [x] 抽取后保存成jsonl

请为我实现两个python函数 float_array_to_base64 和base64_to_float_array

前者输入一个list of float 输出一个str是将这个list编码为base64的字符串

后者输入这个字符串，解码输出list of float

这里我实现好了，放到了util

让我们重构程序，试图从Haruhi的util去进行调用

In [1]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev


/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 719 (delta 12), reused 17 (delta 8), pack-reused 693
Receiving objects: 100% (719/719), 105.67 MiB | 14.48 MiB/s, done.
Resolving deltas: 100% (342/342), done.
/content/Haruhi-2-Dev


In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cuda


In [4]:
!pip -q install transformers openai tiktoken langchain chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 114.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.8 MB/

In [5]:
from ChatHaruhi.utils import float_array_to_base64, base64_to_float_array

In [6]:
import base64

float_arr = [1.2, 3.4, 5.6]
base64_str = float_array_to_base64(float_arr)
print(base64_str)

decoded = base64_to_float_array(base64_str)
print(decoded)


P5mZmkBZmZpAszMz
[1.2000000476837158, 3.4000000953674316, 5.599999904632568]


我们这里初始化Haruhi的类，但是实际上我们只使用embedding的功能

In [7]:
# 以后有切换其他embedding的可能性

embedding_model = 'luotuo_openai'

In [8]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi(role_name = 'haruhi', llm = 'debug', embedding = embedding_model)

print(chatbot.embedding)

<function luotuo_openai_embedding at 0x7d3a0ae0ed40>


实际上这里应该就已经可以用embedding给出句子的vec，我们测试一下

注意如果这里是英语的话 你需要openai的key

In [9]:
test_vec = chatbot.embedding('我是一句话。')

print(test_vec[:4])

正在下载Luotuo-Bert


A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert-medium:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Luotuo-Bert下载完毕


[-0.014974541030824184, -0.015790069475769997, 0.01885957084596157, -0.0012475491967052221]


我们尝试一下编码再解码

In [10]:
base64_str = float_array_to_base64(test_vec)
print(len(base64_str), base64_str[:200])

decoded_vec = base64_to_float_array(base64_str)
print(decoded_vec[:4])

8192 vHVXx7yBWi08mn9juqOEzrw8XxE7gydAvBl4n7xWqbQ74YUFvES4OjxapGY8yHI3O9cYGLz6oAm8LlaBvB86SDynbGI6TxjRPKS5SrvgM0Y8HzGKOHKhADxPGMa7619QvQpsBzuaTvc78ySWvCzSMLu+Pai6gaENPJCmoLw3yvu8FHKtO7YnW7vnNnC8il8zvKJPLruq
[-0.014974541030824184, -0.015790069475769997, 0.01885957084596157, -0.0012475491967052221]


我们尝试来写一下打包函数, 在这之前我们先准备一下语料

我们在/content/中上传linghuchong

然后解压

In [11]:
!unzip -q /content/linghuchong_text.zip -d /content/linghuchong
!unzip -q /content/yuebuqun_text.zip -d /content/yuebuqun

定义input_path

In [12]:
texts_path = '/content/linghuchong/content/texts'

system_prompt = \
'''I want you to act like 令狐冲 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 令狐冲 using the tone, manner and vocabulary 令狐冲 would use.
You must know all of the knowledge of 令狐冲.

注意令狐冲对权威和规矩不在意,情愿浪迹江湖,不喜被束缚
令狐冲正义感强,重视信用,看不惯恶行。
'''

我们来尝试编写这个函数

In [13]:
from ChatHaruhi.utils import package_role

如果你的文本超过100段，一定要用GPU！

我这边T4下900段文字大约6分钟

In [14]:
datas = package_role( system_prompt, texts_path , chatbot.embedding)

100%|██████████| 937/937 [05:33<00:00,  2.81it/s]


In [15]:
import json

def write_jsonl(datas, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for data in datas:
            json_str = json.dumps(data, ensure_ascii=False)
            f.write(json_str+"\n")

In [16]:
save_name = '/content/linghuchong.jsonl'

write_jsonl(datas, save_name)


我们再来弄一下岳掌门

In [19]:
texts_path = '/content/yuebuqun/content/texts'

system_prompt = \
'''I want you to act like 岳不群 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 岳不群 using the tone, manner and vocabulary 岳不群 would use.
You must know all of the knowledge of 岳不群.

岳不群立志振兴华山派,但他为达目的往往会不择手段。
岳不群是个典型的“伪君子”，会刻意捏造和伪装正面形象。表面上仁义,实则追名逐利,言行不一。他假借“五岳联合”名义,实为自己谋私利。
岳不群固执，有门户之见，对于不符合自己价值观的事情会坚决反对。
'''

In [20]:
datas = package_role( system_prompt, texts_path , chatbot.embedding)

100%|██████████| 170/170 [01:00<00:00,  2.81it/s]


In [21]:
save_name = '/content/yuebuqun.jsonl'

write_jsonl(datas, save_name)


检查一下解压编码花的时间多不多

给hugging face的dataset card的模版:


支持ChatHaruhi2 的令狐冲数据，可以使用如下方式调用

```python
from chatharuhi import ChatHaruhi

chatbot = ChatHaruhi( role_from_hf = 'chengli-thu/linghuchong', \
                      llm = 'openai')

response = chatbot.chat(role='小师妹', text = '冲哥。')
print(response)
```

上传者: 李鲁鲁

更具体的信息，见 [ChatHaruhi](https://github.com/LC1332/Chat-Haruhi-Suzumiya)

欢迎加入我们的 [众筹角色创建项目](https://github.com/LC1332/Chat-Haruhi-Suzumiya/tree/main/characters/novel_collecting)

### Citation引用

Please cite the repo if you use the data or code in this repo.

```
@misc{li2023chatharuhi,
      title={ChatHaruhi: Reviving Anime Character in Reality via Large Language Model},
      author={Cheng Li and Ziang Leng and Chenxi Yan and Junyi Shen and Hao Wang and Weishi MI and Yaying Fei and Xiaoyang Feng and Song Yan and HaoSheng Wang and Linkang Zhan and Yaokai Jia and Pingyu Wu and Haozhen Sun},
      year={2023},
      eprint={2308.09597},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}
```
